# PMBOK Governance — Agents Notebook

Sections: **[SETUP] [SETUP-ENV] [KERNEL] [TOOLS] [AGENTS] [WIRES] [DEMO]**

This notebook embeds your Mermaid diagram and sets up lightweight, stubbed agents/tools so you can dry-run orchestration without any external services.

## Diagram

```mermaid
flowchart LR
  %% ===================== PMBOK PROJECT GOVERNANCE CHECKPOINTS =====================
  %% Phases aligned to PMBOK; gates & boards shown; steerco/PMO oversight

  %% -------- GOVERNANCE ORG --------
  subgraph GOV["Governance Bodies"]
    STEER[Steering Committee (SteerCo)]
    PMO[PMO / Portfolio Board]
    CCB[Change Control Board (CCB)]
    QA[Quality/Audit]
    FIN[Finance]
  end

  %% -------- INITIATING --------
  subgraph INIT["Initiating"]
    G0{{Gate 0: Idea Intake\n• Problem/Opportunity\n• Sponsor Identified}}
    CHARTER[Project Charter Approved\n• Objectives, Scope, Success Criteria\n• High-level Risks & Budget]
    STAKE[Stakeholder Register\n• RACI draft]
  end

  %% -------- PLANNING --------
  subgraph PLAN["Planning"]
    G1{{Gate 1: Plan Approval\n• Baseline Scope/Schedule/Cost\n• Governance Plan}}
    MGMTPLN[Integrated PM Plan\n• Scope/WBS • Schedule • Cost • Quality\n• Risk • Comms • Procurement • Resources]
    GOVPLAN[Governance Plan\n• Decision Rights • Escalation • Reporting Cadence]
    RISKPLAN[Risk Register & Response Plan]
    PROCU[Procurement Strategy / Make-Buy]
  end

  %% -------- EXECUTING --------
  subgraph EXEC["Executing"]
    G2{{Gate 2: Readiness\n• Team/Contracts Ready\n• Risks & Controls in place}}
    KICK[Kickoff + Comms Plan Live]
    DELIV[Deliverables in Progress]
    CHG[Change Requests → CCB]
  end

  %% -------- MONITOR & CONTROL --------
  subgraph MON["Monitoring & Controlling"]
    PR[Periodic Reviews\n• Status • Variance (SV/CV)\n• Forecast (EAC/ETC)]
    QG{{Quality Gate(s)\n• Entry/Exit Criteria\n• Test Evidence}}
    RREV[Risk Reviews\n• New/Residual Risk • Reserves]
    FINREV[Financial Review\n• Burn vs Budget • Benefits Outlook]
    AUDIT[Audit/Assurance Checks]
  end

  %% -------- CLOSING --------
  subgraph CLOSE["Closing"]
    G3{{Gate 3: Acceptance\n• Scope Verified • UAT Sign-off}}
    TRANS[Transition to Operations\n• Support Model • SLAs]
    LL[Lessons Learned • Retrospective]
    BR[Benefits Realization Plan & Handover]
    ARCH[Archive Artifacts • Contracts Closed]
  end

  %% ===================== FLOWS =====================
  STEER --- PMO
  PMO --- CCB
  PMO --- QA
  PMO --- FIN

  G0 --> CHARTER --> STAKE --> G1
  G1 --> MGMTPLN --> GOVPLAN --> RISKPLAN --> PROCU --> G2
  G2 --> KICK --> DELIV --> PR
  CHG --> CCB --> PR

  PR --> QG --> PR
  PR --> RREV --> PR
  PR --> FINREV --> PR
  QA --> AUDIT --> PR

  PR -->|Meets tolerances| G3
  PR -->|Breaches tolerances| STEER
  STEER -->|Direction/Escalation| PR

  G3 --> TRANS --> LL --> BR --> ARCH

  %% ===================== NOTES =====================
  %% - Gates (G0–G3) require documented criteria & sign-offs.
  %% - Status reviews show EVM (SV/CV), forecast (EAC), risks, issues, decisions.
  %% - CCB governs scope/time/cost changes; PMO tracks portfolio alignment.
  %% - Quality gates enforce definition of done/ready; Audit samples compliance.
  %% - Closing captures benefits handover and lessons for the portfolio.

  %% ===================== STYLES =====================
  classDef phase fill:#ecfeff,stroke:#06b6d4,stroke-width:2px,color:#134e4a;
  classDef gate  fill:#fff7ed,stroke:#fb923c,stroke-width:2px,color:#7c2d12;
  classDef gov   fill:#f5f3ff,stroke:#8b5cf6,stroke-width:2px,color:#4c1d95;

  class GOV,STEER,PMO,CCB,QA,FIN gov
  class INIT,PLAN,EXEC,MON,CLOSE phase
  class G0,G1,G2,G3,QG gate
```

## %% [SETUP]
Basic stdlib-only setup; no external installs.

In [ ]:
# %% [SETUP]
from __future__ import annotations
import os, json, time, uuid, asyncio, random
from dataclasses import dataclass, field
print("Setup complete.")

## %% [SETUP-ENV]
Optional Azure OpenAI env (not required for this demo).

In [ ]:
# %% [SETUP-ENV]
import os, getpass
os.environ.setdefault('AZURE_OPENAI_ENDPOINT', 'https://example.openai.azure.com')
os.environ.setdefault('AZURE_OPENAI_DEPLOYMENT', 'gpt-35-turbo')
os.environ.setdefault('AZURE_OPENAI_API_VERSION', '2024-10-21')
if not os.getenv('AZURE_OPENAI_API_KEY'):
    try:
        os.environ['AZURE_OPENAI_API_KEY'] = getpass.getpass('Enter AZURE_OPENAI_API_KEY (hidden, optional): ').strip()
    except Exception:
        pass
print('Azure OpenAI env staged (key optional).')

## %% [KERNEL]
Minimal agent/kernel primitives.

In [ ]:
# %% [KERNEL]
@dataclass
class ToolResult:
    name: str
    ok: bool
    data: dict

class Tool:
    def __init__(self, name, fn):
        self.name = name
        self._fn = fn
    def invoke(self, **kwargs):
        try:
            return ToolResult(self.name, True, self._fn(**kwargs))
        except Exception as e:
            return ToolResult(self.name, False, {"error": str(e)})
    def cost_estimate(self, **kwargs):
        payload_size = sum(len(str(v)) for v in kwargs.values())
        return max(1, payload_size // 20)

class Planner:
    def __init__(self, allow_tools=None):
        self.allow_tools = set(allow_tools or [])
    def plan(self, prompt: str, context: dict):
        intent = context.get("intent", "general")
        steps = ["precheck", "maybe_tool", "draft", "postcheck"]
        return {"intent": intent, "steps": steps, "tool_budget": 200}
    def select_tools(self, intent: str):
        return [t for t in self.allow_tools]

class PolicyEngine:
    def precheck(self, prompt: str, plan: dict):
        return {"allow": True, "constraints": {"max_tokens": 400}}
    def postcheck(self, answer: str, citations=None):
        return {"allow": True, "redact": False}

class Agent:
    def __init__(self, name: str, tools: dict[str, Tool]):
        self.name = name
        self.id = str(uuid.uuid4())[:8]
        self.tools = tools
        self.planner = Planner(allow_tools=set(tools.keys()))
        self.policy = PolicyEngine()
    def available_tools(self):
        return list(self.tools.keys())
    def call(self, tool_name: str, **kwargs):
        t = self.tools.get(tool_name)
        if not t:
            return {"error": f"tool '{tool_name}' not found"}
        res = t.invoke(**kwargs)
        return res.data if res.ok else res.data
    async def run(self, prompt: str, intent: str = "general"):
        plan = self.planner.plan(prompt, {"intent": intent})
        pre = self.policy.precheck(prompt, plan)
        if not pre.get("allow", True):
            return {"answer": "[refused by policy]", "citations": [], "confidence": 0.0}
        answer = f"[{self.name}] Drafted response for intent='{intent}': {prompt[:160]}"
        post = self.policy.postcheck(answer, [])
        conf = round(random.uniform(0.75, 0.98), 2)
        return {"answer": answer, "citations": [], "confidence": conf}

## %% [TOOLS]
Stubbed tool adapters for governance use-cases.

In [ ]:
# %% [TOOLS]
def t_policy_check(area:str, gate:str):
    return {"area": area, "gate": gate, "verdict": "pass", "notes": "Criteria met"}
def t_quality_gate(artifact:str, criteria:str):
    return {"artifact": artifact, "criteria": criteria, "evidence": "tests+docs", "verdict": "pass"}
def t_risk_review(phase:str):
    return {"phase": phase, "top_risks": ["scope creep", "vendor delay"], "status": "reviewed"}
def t_finance_review(period:str):
    return {"period": period, "burn_vs_budget": "within 3%", "forecast": "on-track"}
def t_audit_check(scope:str):
    return {"scope": scope, "sample_rate": "10%", "findings": "none"}
TOOLS = {
    "policy_check": Tool("policy_check", t_policy_check),
    "quality_gate": Tool("quality_gate", t_quality_gate),
    "risk_review": Tool("risk_review", t_risk_review),
    "finance_review": Tool("finance_review", t_finance_review),
    "audit_check": Tool("audit_check", t_audit_check),
}
print("Tools ready:", ", ".join(TOOLS.keys()))

## %% [AGENTS]
Agents aligned to PMBOK phases and governance bodies.

In [ ]:
# %% [AGENTS]
agent_governance = Agent("SteerCo/PMO Agent", {k: TOOLS[k] for k in ("policy_check","finance_review","audit_check")})
agent_planning   = Agent("Planning Agent",     {k: TOOLS[k] for k in ("policy_check","risk_review","quality_gate")})
agent_execution  = Agent("Execution Agent",    {k: TOOLS[k] for k in ("quality_gate","risk_review")})
agent_monitoring = Agent("Monitoring Agent",   {k: TOOLS[k] for k in ("risk_review","finance_review","audit_check")})
agent_closing    = Agent("Closing Agent",      {k: TOOLS[k] for k in ("quality_gate","audit_check")})
agent_change     = Agent("Change Control Agent",{k: TOOLS[k] for k in ("policy_check","quality_gate")})

AGENT_INDEX = {
    "governance": agent_governance,
    "planning": agent_planning,
    "execution": agent_execution,
    "monitoring": agent_monitoring,
    "closing": agent_closing,
    "change": agent_change,
}
print("Agents:", ", ".join(f"{k}:{v.name}" for k,v in AGENT_INDEX.items()))

## %% [WIRES]
Route key checkpoints to agents + wiring counter.

In [ ]:
# %% [WIRES]
ROUTES = {
    "Gate 0": "governance",
    "Gate 1": "planning",
    "Gate 2": "execution",
    "Gate 3": "closing",
    "Periodic Review": "monitoring",
    "Change Request": "change",
}

def validate_wiring():
    problems = []
    for checkpoint, key in ROUTES.items():
        agent = AGENT_INDEX.get(key)
        if not agent:
            problems.append(f"{checkpoint} -> missing agent key '{key}'")
            continue
        if not agent.available_tools():
            problems.append(f"{checkpoint} -> {agent.name} has no available tools")
    return problems

total_wires = len(ROUTES)
distinct_agents = len(set(ROUTES.values()))
unreferenced_agents = sorted(set(AGENT_INDEX.keys()) - set(ROUTES.values()))
targets_by_agent = {}
for cp, key in ROUTES.items():
    targets_by_agent.setdefault(key, []).append(cp)

issues = validate_wiring()
print(f"Wires: {total_wires} (distinct agents: {distinct_agents})")
for agent_key, cps in targets_by_agent.items():
    agent_name = AGENT_INDEX[agent_key].name
    print(f"  - {agent_name} <- {len(cps)} checkpoint(s): {', '.join(cps)}")
if unreferenced_agents:
    print("Unreferenced agents:", ", ".join(unreferenced_agents))
print("Wiring OK" if not issues else "Wiring issues:\n- " + "\n- ".join(issues))

## %% [DEMO]
Notebook-safe async demo (no `asyncio.run()` loop issues).

In [ ]:
# %% [DEMO]
# Simulated scenarios across checkpoints; tools are stubbed, LLM is a toy draft.
import asyncio, time

samples = [
    ("Gate 0", "Idea intake: AI chatbot for HR FAQs."),
    ("Gate 1", "Approve baseline plan and governance."),
    ("Gate 2", "Readiness check before kickoff."),
    ("Periodic Review", "Show SV/CV with EAC forecast."),
    ("Change Request", "Add scope for multilingual support."),
    ("Gate 3", "Acceptance and UAT sign-off."),
]

async def demo_run():
    t0 = time.time()
    outputs = []
    for checkpoint, text in samples:
        key = ROUTES[checkpoint]
        agent = AGENT_INDEX[key]
        tool_result = None
        if key == "planning":
            tool_result = agent.call("risk_review", phase="planning")
        elif key == "execution":
            tool_result = agent.call("quality_gate", artifact="build#123", criteria="DoR/DoD")
        elif key == "monitoring":
            tool_result = agent.call("finance_review", period="Q1")
        elif key == "change":
            tool_result = agent.call("policy_check", area="scope", gate="CCB")
        elif key == "closing":
            tool_result = agent.call("audit_check", scope="UAT evidence")
        else:
            tool_result = agent.call("policy_check", area="governance", gate=checkpoint)

        llm_out = await agent.run(text, intent=key)
        outputs.append({
            "checkpoint": checkpoint,
            "agent": agent.name,
            "tool_result": tool_result,
            "llm_result": llm_out["answer"][:220] + ("..." if len(llm_out["answer"])>220 else ""),
            "confidence": llm_out["confidence"],
        })
    elapsed_ms = int((time.time() - t0)*1000)
    return {"elapsed_ms": elapsed_ms, "runs": outputs}

try:
    loop = asyncio.get_running_loop()
    try:
        result = await demo_run()
    except SyntaxError:
        import nest_asyncio; nest_asyncio.apply()
        result = loop.run_until_complete(demo_run())
except RuntimeError:
    result = asyncio.run(demo_run())

print("Elapsed (ms):", result["elapsed_ms"])
for r in result["runs"]:
    print((
        f"\nCheckpoint: {r['checkpoint']} -> Agent: {r['agent']}"
        f"\nTool: {r['tool_result']}"
        f"\nLLM:  {r['llm_result']} (conf={r['confidence']:.2f})"
    ).rstrip())